In [1]:
import io
import re
import itertools

import geojson
import netCDF4
import pandas
import numpy as np


In [2]:
ds = netCDF4.Dataset('../data/transect.nc')

ids = ds.variables['id'][:]

id_ = 7003900

def transect2features(id_):
    transect_idx = np.searchsorted(ids, id_)
    
    variables = {
        'lat': {"var": 'lat', "slice": np.s_[transect_idx, :]},
        'lon': {"var": 'lon', "slice": np.s_[transect_idx, :]}

    }
    data = {}
    for var, props in variables.items():
        data[var] = ds.variables[props['var']][props['slice']]

    #for t, row in zip(data['t'], data['z']):
    coords = pandas.DataFrame(data=dict(
        lon0=data['lon'][:1], 
        lat0=data['lat'][:1],
        lon1=data['lon'][-1:],
        lat1=data['lat'][-1:]
    ))
    coordinates = [coords.dropna().values[0][0:2].tolist(),coords.dropna().values[0][2:4].tolist()]
    geometry = geojson.LineString(coordinates=coordinates)
    properties = {
        "id": "{}".format(id_)
    }
    yield geojson.Feature(id="{}".format(id_), geometry=geometry, properties=properties)


In [3]:
import itertools
import more_itertools

In [4]:
import tqdm

features = []

for id_ in tqdm.tqdm_notebook(ids):
    features.extend(transect2features(id_))



In [5]:
for i, chunk in enumerate(more_itertools.chunked(features, 10000)):
    collection = geojson.FeatureCollection(chunk)
    with open('jarkus4mapbox.json'.format(i), 'w') as f:
        geojson.dump(collection, f)


In [6]:
%pdb off


Automatic pdb calling has been turned OFF
